# Unsupervised Learning

## Gensim LDA (Latent Dirichlet Allocation)

In [22]:
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords

import sys
import os

import gensim
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.matutils import corpus2csc

from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

import tqdm

import tomotopy as tp

from GPyM_TM.GSDMM import DMM
from GPyM_TM import GPM

from preprocess import preprocess, tokenizer, stopword_list

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
data_root = '../data/cvt/'
df = pd.read_csv(os.path.join(data_root, 'covidvaccine.csv'))
df

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,hashtag_count
0,Ashwin,India 🇮🇳,I'm Just A Man On a Mission For My Vision!\n\n...,2018-01-08 06:01:00,89.0,3108.0,6780.0,0,2020-08-09 19:15:00,Huge Thanks And Best Wishes With @SerumInstInd...,covisheild,Twitter for Android,1
1,K. Kartik Rao,Mumbai and Nagpur India,Actor 🎭 \n\nhttps://t.co/u7GZfoWBDt\n\nSubscri...,2016-12-10 16:06:00,225.0,257.0,25362.0,0,2020-08-09 20:02:00,Any update of #CovidVaccine ????,covidvaccine,Twitter for Android,1
2,Mac Patrick (Road Hater) Henry,German Bundesrepublik,Liberty lover. \nServing the people of the 🇺🇸....,2014-01-11 19:47:00,338.0,277.0,28305.0,0,2020-08-09 21:21:00,From a private frontline COVID nurses group to...,covid19 covidvaccinerace covidvaccine pandemic,Twitter Web App,4
3,lydia Bell,Earth,"Mummy, animal lover , energy worker , awake in...",2009-03-23 01:09:00,2038.0,1105.0,63422.0,0,2020-08-09 21:29:00,So you want the whole world to take a #covidva...,covidvaccine,Twitter for iPhone,1
4,Lou Ann Donovan,"Kansas, USA",Live Young Look Young BE Young! Certified Inte...,2009-03-17 05:42:00,4365.0,4581.0,2505.0,0,2020-08-09 22:55:00,Watch to learn about the Phase 3 clinical tria...,covid19 covidvaccine covidclinicaltrial,Twitter for iPhone,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154533,Script Your Future '21,DMV,HowardU and Script Your Future has joined forc...,2021-01-31 18:40:41,15.0,42.0,4.0,0,2021-02-28 23:55:25,* Continue with routine preventive measures (s...,NaN,Twitter Web App,0
154534,eastvillagetwt,"East Village, New York, NY","Politics, the environment, science, culture, t...",2011-08-27 19:24:17,1968.0,5001.0,136110.0,0,2021-02-28 23:56:37,COVID: California Suspends System Of Shared Va...,covid19 covidvaccine,Twitter for iPhone,2
154535,Yogita,"New York, NY",Primum non nocere 💕👩🏽‍⚕️ | Tweets are my own. ...,2015-08-07 01:48:57,17.0,58.0,1194.0,0,2021-02-28 23:57:00,Today I got my first dose of the #COVIDVaccine...,covidvaccine,Twitter for Android,1
154536,DrZ,USA,NaN,2016-03-23 13:00:36,54.0,4.0,4.0,0,2021-02-28 23:57:06,Here's Where to Receive a COVID Vaccine in Ill...,illinois covidvaccine,Buffer,2


In [24]:
df['text'] = df['text'].apply(lambda x: preprocess(x))

df

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,hashtag_count
0,Ashwin,India 🇮🇳,I'm Just A Man On a Mission For My Vision!\n\n...,2018-01-08 06:01:00,89.0,3108.0,6780.0,0,2020-08-09 19:15:00,huge thanks best wishes seruminstindia adarpoo...,covisheild,Twitter for Android,1
1,K. Kartik Rao,Mumbai and Nagpur India,Actor 🎭 \n\nhttps://t.co/u7GZfoWBDt\n\nSubscri...,2016-12-10 16:06:00,225.0,257.0,25362.0,0,2020-08-09 20:02:00,update covidvaccine,covidvaccine,Twitter for Android,1
2,Mac Patrick (Road Hater) Henry,German Bundesrepublik,Liberty lover. \nServing the people of the 🇺🇸....,2014-01-11 19:47:00,338.0,277.0,28305.0,0,2020-08-09 21:21:00,private frontline covid nurses group today cov...,covid19 covidvaccinerace covidvaccine pandemic,Twitter Web App,4
3,lydia Bell,Earth,"Mummy, animal lover , energy worker , awake in...",2009-03-23 01:09:00,2038.0,1105.0,63422.0,0,2020-08-09 21:29:00,want whole world take covidvaccine s fast trac...,covidvaccine,Twitter for iPhone,1
4,Lou Ann Donovan,"Kansas, USA",Live Young Look Young BE Young! Certified Inte...,2009-03-17 05:42:00,4365.0,4581.0,2505.0,0,2020-08-09 22:55:00,watch learn phase clinical trial covid vaccine...,covid19 covidvaccine covidclinicaltrial,Twitter for iPhone,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154533,Script Your Future '21,DMV,HowardU and Script Your Future has joined forc...,2021-01-31 18:40:41,15.0,42.0,4.0,0,2021-02-28 23:55:25,continue routine preventive measures vaccinati...,NaN,Twitter Web App,0
154534,eastvillagetwt,"East Village, New York, NY","Politics, the environment, science, culture, t...",2011-08-27 19:24:17,1968.0,5001.0,136110.0,0,2021-02-28 23:56:37,covid california suspends system shared vaccin...,covid19 covidvaccine,Twitter for iPhone,2
154535,Yogita,"New York, NY",Primum non nocere 💕👩🏽‍⚕️ | Tweets are my own. ...,2015-08-07 01:48:57,17.0,58.0,1194.0,0,2021-02-28 23:57:00,today got first dose covidvaccine super thankful,covidvaccine,Twitter for Android,1
154536,DrZ,USA,NaN,2016-03-23 13:00:36,54.0,4.0,4.0,0,2021-02-28 23:57:06,heres receive covid vaccine illinois illinois ...,illinois covidvaccine,Buffer,2


In [25]:
df['text']

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0         huge thanks best wishes seruminstindia adarpoo...
1                                       update covidvaccine
2         private frontline covid nurses group today cov...
3         want whole world take covidvaccine s fast trac...
4         watch learn phase clinical trial covid vaccine...
                                ...                        
154533    continue routine preventive measures vaccinati...
154534    covid california suspends system shared vaccin...
154535     today got first dose covidvaccine super thankful
154536    heres receive covid vaccine illinois illinois ...
154537    utah slc sold walmart vaccine first dose book ...
Name: text, Length: 154538, dtype: object

In [26]:
# Prepare tokenized text
text_tokens = []
docs = [tokenizer(doc) for doc in list(df['text'])]
for doc in docs:
    text_tokens.append([token.orth_ for token in doc])

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
def compute_coherence_values_gsdmm(corpus, k, a, b):
    dmm = DMM(corpus, k, alpha=a, beta=b, iters=10) # Initialize the object.

    dmm.topicAssigmentInitialise() # Performs the inital document assignments and counts
    dmm.inference()

    psi, theta, selected_psi, selected_theta = dmm.worddist() # Determines and stores the psi, theta and selected_psi and selected_theta values

    final_assignments = dmm.writeTopicAssignments() # Records the final topic assignments for the documents

    coherence_topwords = dmm.writeTopTopicalWords(final_assignments) # Record the top words for each document

    score = dmm.coherence(coherence_topwords, len(final_assignments)) # Calculates and stores the coherence

    return score

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
already_run = False

# Topics range (length = 10)
min_topics = 2
max_topics = 18
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter (length = 5)
alpha = list(np.arange(0.2, 1, 0.2))

# Beta parameter (length = 5)
beta = list(np.arange(0.2, 1, 0.2))

model_results = {
    'Topics': [],
    'Alpha': [],
    'Beta': [],
    'Coherence': []
}

if not already_run:
    # Can take a long time to run
    pbar = tqdm.tqdm(total=2200)
    
    # Iterate through number of topics
    for k in topics_range:
        # Iterate through alpha values
        for a in alpha:
            # Iterate through beta values
            for b in beta:
                # Get the coherence score for the given parameters
                cv = compute_coherence_values_gsdmm(corpus=text_tokens,k=k, a=a, b=b)
                #print(umass,cv)
                # Save the model results
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)


                pbar.update(1)
    coherence_results = pd.DataFrame(model_results)
    coherence_results.to_csv('../out/gsdmm_tuning_results.csv', index=False)
    pbar.close()
else:
    coherence_results = pd.read_csv('../out/gsdmm_tuning_results.csv')

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  0%|          | 0/2200 [00:46<?, ?it/s]


corpus=154538, words=91498, K=2, a=0.200000, b=0.200000, nTopWords=10, iters=10
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
[0 1]
covidvaccine covid vaccine get today first got dose s people 
covidvaccine covid vaccine vaccines coronavirus s people pfizer india first 




  0%|          | 1/2200 [01:14<45:46:55, 74.95s/it]

average topic:  -36.567735530468326
corpus=154538, words=91498, K=2, a=0.200000, b=0.400000, nTopWords=10, iters=10
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
[0 1]
covidvaccine covid vaccine vaccines people s coronavirus get pfizer india 
covidvaccine covid vaccine get today first got dose s people 




  0%|          | 2/2200 [02:31<46:19:54, 75.88s/it]

...

average topic:  -43.05753911791934
corpus=154538, words=91498, K=16, a=0.800000, b=0.600000, nTopWords=10, iters=10
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
white_large_square syringe progress dose black_large_square [ ] second first covidvaccine 
education covidvaccine recorded song covid big _ ape redfored top 
largestvaccinedrive covidvaccine vaccine shares dr experience black district chaos roll 
covidvaccine dr warns vax wakefield genetic irreversible modification rabbi covid 
maximebernier billtufts clinicalmemes friendsoscience goodtexture mainetain massgovernor terencecorcoran evankirstel nfpublication 
covidvaccine time arranging shortest bangladeshis sheikhhasina hpm thankyou lucky ge 
covidvaccine vaccine covid got dose today first get second s 
live happs pm us via join back better eu instead 
covidvaccine covid vaccine get people s vacc



  6%|▌         | 127/2200 [9:17:27<279:31:53, 485.44s/it]

average topic:  -48.16102142543907
corpus=154538, words=91498, K=16, a=0.800000, b=0.800000, nTopWords=10, iters=10
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
white_large_square syringe rabbi dose progress black_large_square read article latest trends 
trump s many fired suspended reversed eo week st biden 
largestvaccinedrive shares experience district shri vaccine dr way black london 
mrna nvax bntx cvac adpt alny atra ebs arct vxrt 
test face_with_thermometer case done discharged doubling person_running skull_and_crossbones days death 
time covidvaccine arranging shortest bangladeshis sheikhhasina hpm thankyou lucky ge 
covidvaccine covid dr masks care b nt coronavirus govnedlamont comorbidities 
recorded song pandemic coronavirus original isolated live covidvaccine whether misinformation 
worse covid men shall evil covidvaccine wax seducers timoth



  6%|▌         | 128/2200 [9:25:58<152:41:46, 265.30s/it]

average topic:  -43.96227041464071


In [31]:
# Sort coherence_results
sorted_coherence_results = coherence_results.sort_values(by=['Coherence'], ascending=False)
sorted_coherence_results

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topics,Alpha,Beta,Coherence
7,2,0.4,0.8,-35.734815
9,2,0.6,0.4,-35.916374
11,2,0.6,0.8,-36.031520
15,2,0.8,0.8,-36.134665
14,2,0.8,0.6,-36.265265
...,...,...,...,...
75,10,0.6,0.8,-50.578203
51,8,0.2,0.8,-51.685738
59,8,0.6,0.8,-51.702597
39,6,0.4,0.8,-52.451347


In [32]:
# View the top coherence result
best_hyperparams = sorted_coherence_results.iloc[0]
best_hyperparams

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topics        2.000000
Alpha         0.400000
Beta          0.800000
Coherence   -35.734815
Name: 7, dtype: float64

In [38]:
dmm = DMM(text_tokens, 
          int(best_hyperparams['Topics']), 
          alpha=best_hyperparams['Alpha'], 
          beta=best_hyperparams['Beta'],
          iters=10)

dmm.topicAssigmentInitialise() # Performs the inital document assignments and counts
dmm.inference()

psi, theta, selected_psi, selected_theta = dmm.worddist() # Determines and stores the psi, theta and selected_psi and selected_theta values

final_assignments = dmm.writeTopicAssignments() # Records the final topic assignments for the documents

coherence_topwords = dmm.writeTopTopicalWords(final_assignments) # Record the top words for each document

score = dmm.coherence(coherence_topwords, len(final_assignments)) # Calculates and stores the coherence

print("Final Assigments: ", final_assignments)

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


corpus=154538, words=91498, K=2, a=0.400000, b=0.800000, nTopWords=10, iters=10
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
[0 1]
covidvaccine covid vaccine vaccines people s coronavirus get pfizer india 
covidvaccine covid vaccine get today first got dose s people 
average topic:  -36.2781234850246
Final Assigments:  [0 1]


In [23]:
mdl.summary()

topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq

prepared_data = pyLDAvis.prepare(
    topic_term_dists, 
    doc_topic_dists, 
    doc_lengths, 
    vocab, 
    term_frequency
)

pyLDAvis.save_html(prepared_data, '../out/ldavis.html')

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<Basic Info>
| LDAModel (current version: 0.11.1)
| 154535 docs, 1531044 words
| Total Vocabs: 76786, Used Vocabs: 76786
| Entropy of words: -7.72703
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.14847
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 20 (the number of topics between 1 ~ 32767)
| alpha: [0.01] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.91 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 1778162353 (random seed)
| trained in version 0.11.1
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [1.14122406e-04 2.67013209e-04 4.92286636e-03

In [24]:
rows = {}

for i in range(int(best_hyperparams['Topics'])):
    topic_words = mdl.get_topic_words(i)
    rows['Topic #' + str(i)] = [w for w, p in topic_words]

topic_words = pd.DataFrame.from_dict(rows, orient='index',
                       columns=['Keyword #1', 'Keyword #2', 'Keyword #3', 'Keyword #4', 'Keyword #5',
                               'Keyword #6', 'Keyword #7', 'Keyword #8', 'Keyword #9', 'Keyword #10'])
topic_words

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Keyword #1,Keyword #2,Keyword #3,Keyword #4,Keyword #5,Keyword #6,Keyword #7,Keyword #8,Keyword #9,Keyword #10
Topic #0,foggi,premium,woven,flamingo,static,layer,glass,filter,mrjamesob,insert
Topic #1,rabbi,zorger,mosh,jerusalem,satmar,wolf,rule,chief,research,templat
Topic #2,year,postpon,pleas,tokyoolymp,parisolymp,laolymp,hello,nbcnew,nbcolymp,janon
Topic #3,genet,modif,wakefield,irrevers,warn,vax,dr,bme_dementia,meriyaadainc,icai
Topic #4,footwearmal,_,happynewmonth,thesaganadeclar,shop,iqoo,nspector,timesofindia,hci_ottawa,rickanderson
Topic #5,sportstalkfla,newstalkflorida,ussportsacademi,jwmediadc,wordmandc,sportsbiz,sportsnew,sport,upwards_button,nhl
Topic #6,maximeberni,vernon,friendsosci,coleman,mb,billtuft,goodtextur,clinicalmem,dsc,mainetain
Topic #7,educ,redfor,ape,tbat,big,top,headlin,banana,print,redbubbl
Topic #8,mondaynightibd,dcharabati,dr_fcalderaibd,mregueiromd,abbvi,edwardloftu,mchiorean,ibd_afzali,moss_md,civilizedppl
Topic #9,face_with_thermomet,skull_and_crossbon,person_run,discharg,doubl,raised_fist_medium,done,dark_skin_ton,_,btc


In [25]:
token_counts = mdl.get_count_by_topics()
topic_words.insert(0, 'Token Counts', token_counts)
topic_words

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Token Counts,Keyword #1,Keyword #2,Keyword #3,Keyword #4,Keyword #5,Keyword #6,Keyword #7,Keyword #8,Keyword #9,Keyword #10
Topic #0,350,foggi,premium,woven,flamingo,static,layer,glass,filter,mrjamesob,insert
Topic #1,816,rabbi,zorger,mosh,jerusalem,satmar,wolf,rule,chief,research,templat
Topic #2,11105,year,postpon,pleas,tokyoolymp,parisolymp,laolymp,hello,nbcnew,nbcolymp,janon
Topic #3,2117,genet,modif,wakefield,irrevers,warn,vax,dr,bme_dementia,meriyaadainc,icai
Topic #4,311,footwearmal,_,happynewmonth,thesaganadeclar,shop,iqoo,nspector,timesofindia,hci_ottawa,rickanderson
Topic #5,157,sportstalkfla,newstalkflorida,ussportsacademi,jwmediadc,wordmandc,sportsbiz,sportsnew,sport,upwards_button,nhl
Topic #6,226,maximeberni,vernon,friendsosci,coleman,mb,billtuft,goodtextur,clinicalmem,dsc,mainetain
Topic #7,2973,educ,redfor,ape,tbat,big,top,headlin,banana,print,redbubbl
Topic #8,1035,mondaynightibd,dcharabati,dr_fcalderaibd,mregueiromd,abbvi,edwardloftu,mchiorean,ibd_afzali,moss_md,civilizedppl
Topic #9,5898,face_with_thermomet,skull_and_crossbon,person_run,discharg,doubl,raised_fist_medium,done,dark_skin_ton,_,btc


In [26]:
topic_words.to_csv('../out/lda_topic_words.csv', index=True)

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
